# 1. NUCLEI SEGMENTATION - STARDIST (PYTHON)


In [ ]:
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
import matplotlib.image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import seaborn as sns
import PIL
import tifffile
import sys
import pickle
#import scanpy as sc
#import os
#import bin2cell as b2c
#import cv2




In [ ]:
# functions

def custom_normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """Percentile-based image normalization."""

    mi = np.array([[[134.]]])#np.percentile(x,pmin,axis=axis,keepdims=True)
    ma = np.percentile(x,pmax,axis=axis,keepdims=True)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)


def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

    try:
        import numexpr
        x = numexpr.evaluate("(x - mi) / ( ma - mi + eps )")
    except ImportError:
        x =                   (x - mi) / ( ma - mi + eps )

    if clip:
        x = np.clip(x,0,1)

    return x


In [ ]:
# INPUT DATA

# img hires
PIL.Image.MAX_IMAGE_PIXELS = 1718032108
img = tifffile.imread(source_image_path)
img = img[:,:,0:3]

# Histological_image from sample.csv
# dare in input dal csv dei sample il valore di Histological_image in variabile hist_img to choose StarDist model

In [ ]:
# normalizzazione immagine custom

normalized_img = custom_normalize(img)


In [ ]:
# choose stardist model based on image, hist_img vedi input

if hist_img == "H&E":
    model_stardist = "2D_versatile_he"
elif hist_img == "IF":
    model_stardist = "2D_versatile_fluo"

model = StarDist2D.from_pretrained(model_stardist)

In [ ]:
# run stardist model

labels, polys = model.predict_instances_big(
    normalized_img, axes='YXC', block_size=4096, min_overlap=128, context=128,
    normalizer=None, # n_tiles=(4,4,1),
    prob_thresh = .05)

# potenzialmente prob_thresh e nms_thresh (qui default) potrebbero essere parametri da dare in input

In [ ]:
# save labels and polys
#with open(results_folder + sample_name + '_nuclei_labels.pkl', 'wb') as f:  # open a text file
#    pickle.dump(labels, f)
# i labels non dovrebbero servire più

with open(results_folder + sample_name + '_nuclei_polys.pkl', 'wb') as f:  # open a text file
    pickle.dump(polys, f)

# 2. BIN2CELL EXPANSION (PYTHON)

In [ ]:
from stardist import random_label_cmap, _draw_polygons
import pandas as pd
import numpy as np
import scanpy as sc
from scipy import sparse
import bin2cell as b2c
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import cv2
import os
import pickle
import anndata
import geopandas as gpd
from tifffile import imread, imwrite
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
from shapely.affinity import scale
from scipy.spatial.distance import pdist
import scrublet as scr
import itertools
import anndata as ad
from pandas.api.types import CategoricalDtype
import random
import sys


In [ ]:
# functions

def remove_destripe_artifacts(adata):
    idx = adata.X.sum(axis=1).A1 == float('inf')
    print('removing '+str(idx.sum())+' dots')
    return adata[idx==False]

def nuclei_detection(polys, tissue_position_file, adata):
    # Creating a list to store Polygon geometries
    geometries = []

    # Iterating through each nuclei in the 'polys' DataFrame
    for nuclei in range(len(polys['coord'])):

        # Extracting coordinates for the current nuclei and converting them to (y, x) format
        coords = [(y, x) for x, y in zip(polys['coord'][nuclei][0], polys['coord'][nuclei][1])]

        # Creating a Polygon geometry from the coordinates
        geometries.append(Polygon(coords))

    # Creating a GeoDataFrame using the Polygon geometries
    gdf = gpd.GeoDataFrame(geometry=geometries)
    gdf['id'] = [f"ID_{i+1}" for i, _ in enumerate(gdf.index)]

    ########################################################################

    # Load the Spatial Coordinates
    df_tissue_positions=pd.read_parquet(tissue_position_file)

    #Set the index of the dataframe to the barcodes
    df_tissue_positions = df_tissue_positions.set_index('barcode')

    # Create an index in the dataframe to check joins
    df_tissue_positions['index']=df_tissue_positions.index

    # Adding the tissue positions to the meta data
    adata.obs =  pd.merge(adata.obs, df_tissue_positions[['pxl_row_in_fullres', 'pxl_col_in_fullres']], left_index=True, right_index=True)

    # Create a GeoDataFrame from the DataFrame of coordinates
    geometry = [Point(xy) for xy in zip(df_tissue_positions['pxl_col_in_fullres'], df_tissue_positions['pxl_row_in_fullres'])]
    gdf_coordinates = gpd.GeoDataFrame(df_tissue_positions, geometry=geometry)

    #########################################################################

    # Perform a spatial join to check which coordinates are in a cell nucleus
    result_spatial_join = gpd.sjoin(gdf_coordinates, gdf, how='left', predicate='within')

    result_spatial_join["in_tissue"] = np.array(result_spatial_join["in_tissue"]).astype(bool)

    # Identify nuclei associated barcodes and find barcodes that are in more than one nucleus
    result_spatial_join['is_within_polygon'] = ~result_spatial_join['index_right'].isna()
    barcodes_in_overlaping_polygons = pd.unique(result_spatial_join[result_spatial_join.duplicated(subset=['index'])]['index'])
    result_spatial_join['is_not_in_an_polygon_overlap'] = ~result_spatial_join['index'].isin(barcodes_in_overlaping_polygons)

    # Remove barcodes in overlapping nuclei
    barcodes_in_one_polygon =  result_spatial_join[result_spatial_join['is_not_in_an_polygon_overlap'] & result_spatial_join["in_tissue"]] # result_spatial_join['is_within_polygon'] &
    # tengo anche quelli unassigned to any nucleus perchè potrebbero essere in seguito aggregati come citoplasma
    # tanto in bin2cell per l'aggregazione dei nuclei gli 0 non verranno aggregati: Integers, with 0 being unassigned to an object.

    # The AnnData object is filtered to only contain the barcodes that are in non-overlapping polygon regions
    filtered_obs_mask = adata.obs_names.isin(barcodes_in_one_polygon['index'])
    filtered_adata = adata[filtered_obs_mask,:]

    # Add the results of the point spatial join to the Anndata object
    filtered_adata.obs =  pd.merge(filtered_adata.obs, barcodes_in_one_polygon[['index','geometry','id','is_within_polygon','is_not_in_an_polygon_overlap']], left_index=True, right_index=True)

    filtered_adata.obs.id = filtered_adata.obs.id.str.replace('ID_', '', regex=False)
    # filtered_adata.obs = filtered_adata.obs.fillna(0)
    filtered_adata.obs.id = filtered_adata.obs.id.fillna(0)
    filtered_adata.obs.id = filtered_adata.obs.id.astype(str)
    print(f"Number of nuclei detected of hires image: {gdf.shape[0]}\nNumber of nuclei detected on VisiumHD slide: {len(np.unique(filtered_adata.obs.id))}")

    # filter and returns also the geometry dataframe associated to nuclei
    gdf['id'] = gdf['id'].str.replace('ID_', '', regex=False)
    gdf = gdf.loc[gdf['id'].isin(filtered_adata.obs.id)]
    gdf.set_index('id', inplace=True)

    return filtered_adata, gdf


def add_obs_variables(adata, unit, species='Hs'):
    x = adata.copy()
    x.obs['counts_per_' + unit] = np.sum(x.X, axis = 1)
    x.obs['features_per_' + unit] = np.sum(x.X >0, axis = 1)

    if "bin_count" in adata.obs.columns:
        x.obs["bin_count_log"] = np.log10(x.obs["bin_count"])
    x.obs["counts_per_" + unit + "_log"] = np.log10(x.obs["counts_per_" + unit])
    x.obs["features_per_" + unit + "_log"] = np.log10(x.obs["features_per_" + unit])

    if species=='Hs':
        x.var["mt"] = x.var_names.str.startswith("MT-")
    if species=='Mm':
        x.var["mt"] = x.var_names.str.startswith("mt-")
    sc.pp.calculate_qc_metrics(
        x, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
    )
    x.obs.loc[np.isnan(x.obs.pct_counts_mt),'pct_counts_mt'] = 0
    return x



In [ ]:
# INPUT DATA

# polys
#with open(results_folder + sample_name + '_nuclei_labels.pkl', 'rb') as f:  # open a text file
#    labels = pickle.load(f)
with open(results_folder + sample_name + '_nuclei_polys.pkl', 'rb') as f:  # open a text file
    polys = pickle.load(f)

# adata
adata = sc.read_h5ad(output_adata_path)

# parquet
tissue_position_file = # path_name di binned_outputs/square_002um/spatial/tissue_positions.parquet

# Species dal sample.csv in variabile species

In [ ]:
# preprocessing adata

sc.pp.filter_genes(adata, min_cells=3)  # ok filtro sui geni
sc.pp.filter_cells(adata, min_counts=0) # non filtro i pixel perchè andranno aggregati
b2c.destripe(adata,adjust_counts=True)
adata = remove_destripe_artifacts(adata)


In [ ]:
# add segmentation labels to h5ad object, map stardist modeled nuclei on the adata

adata, gdf = nuclei_detection(
    polys = polys, adata=adata,
    tissue_position_file = tissue_position_file
    )
adata.obs.id = adata.obs.id.astype(int)


In [ ]:
# group nuclei based on labels "id"

nuclei_grouped = b2c.bin_to_cell(adata, labels_key="id", spatial_keys=["spatial"])

# annotate
nuclei_grouped = add_obs_variables(nuclei_grouped, "nucleus", species=species)
nuclei_grouped.obs["id"] = np.array(nuclei_grouped.obs.index)
nuclei_grouped.obs['geometry'] = gdf.loc[nuclei_grouped.obs.id]['geometry']
nuclei_grouped.obs['max_diameter'] = [max_diameter(x) for x in nuclei_grouped.obs['geometry']]
nuclei_grouped.obs['zero_mt'] = nuclei_grouped.obs.pct_counts_mt == 0
print('Using ' + str(np.round(nuclei_grouped.X.sum() / adata.X.sum() * 100)) + '% of the total read counts')



In [ ]:
# remove empty nuclei and point geometries

idx = nuclei_grouped.obs.features_per_nucleus >= 2
print('Number of nuclei with at least 2 features: ' + str(np.sum(idx)))
nuclei_grouped = nuclei_grouped[idx]
idx = [geom.geom_type  == 'Polygon' for geom in nuclei_grouped.obs['geometry']]
print('Number of nuclei with polygon geometry: ' + str(np.sum(idx)))
nuclei_grouped = nuclei_grouped[idx]

In [ ]:
# expand nuclei into cells

# reset to "0" (i.e. not assigned) pixels assigned to non-valid nulclei
adata.obs.loc[~adata.obs.id.isin(nuclei_grouped.obs.index.astype(int)),'id'] = 0

# expand nuclei
#adata.obs.id = adata.obs.id.astype(int)   # to expand needs to be numeric
expand = 2      # 2 bins = 4 microns, forse conviene metterlo come parametro in input per fare diversi tentativi di espansione
expansion_label = "id_exp_"+str(expand)
b2c.expand_labels(adata,
                  labels_key='id',
                  expanded_labels_key=expansion_label,
                  max_bin_distance = expand
                 )
adata.obs[expansion_label] = adata.obs[expansion_label].astype(int)



In [ ]:
# group cells based on labels
expanded_nuclei = b2c.bin_to_cell(adata, labels_key=expansion_label, spatial_keys=["spatial"])



In [ ]:
# define new geometries and filter out cells (and corresponding nuclei) with non-Polygon geometry
cell_labels = set(adata.obs[expansion_label])
cell_labels.remove(0)
cell_geoms = {x : cell_geometry(adata[adata.obs[expansion_label] == x].obs['geometry']) for x in cell_labels}
idx = [geom.geom_type  == 'Polygon' for geom in cell_geoms.values()]
print('Number of cells with polygon geometry: ' + str(np.sum(idx)))
nuclei_grouped = nuclei_grouped[idx]
expanded_nuclei = expanded_nuclei[idx]
cell_geoms = {int(x) : cell_geoms[int(x)] for x in expanded_nuclei.obs_names}

In [ ]:
# annotate cells
expanded_nuclei = add_obs_variables(expanded_nuclei, "cell", species=species)
expanded_nuclei.obs["id"] = np.array(expanded_nuclei.obs.index)
expanded_nuclei.obs['geometry'] = [cell_geoms[int(x)] for x in expanded_nuclei.obs.index]
expanded_nuclei.obs['max_diameter'] = [max_diameter(x) for x in expanded_nuclei.obs['geometry']]
expanded_nuclei.obs['zero_mt'] = expanded_nuclei.obs.pct_counts_mt == 0
print('Using ' + str(np.round(expanded_nuclei.X.sum() / adata.X.sum() * 100)) + '% of the total read counts')


In [ ]:
# save geometry on disk
nuclei_grouped_geometry = nuclei_grouped.obs['geometry']
expanded_nuclei_geometry = expanded_nuclei.obs['geometry']
with open(results_folder+'/'+sample_name+'_nuclei_grouped_geometry.pkl', 'wb') as f:  # open a text file,
    pickle.dump(nuclei_grouped_geometry, f),
with open(results_folder+'/'+sample_name+'_expanded_nuclei_geometry.pkl', 'wb') as f:  # open a text file,
    pickle.dump(expanded_nuclei_geometry, f)



In [ ]:
# remove geometries and save h5ad on disk of nuclei and cells
del nuclei_grouped.obs['geometry']
del expanded_nuclei.obs['geometry']
nuclei_grouped.write_h5ad(results_folder+'/'+sample_name+'_nuclei_grouped.h5ad')
expanded_nuclei.write_h5ad(results_folder+'/'+sample_name+'_expanded_nuclei.h5ad')


# 3. RCTD (R) - vedi script

# 4. FILTERS ON DECONVOLUTION RESULTS (R) - vedi script parte 2

# 5. SELECT CELLS/NUCLEI AND DEFINE NEW ADATA (PYTHON)



In [4]:
import pandas as pd
import scanpy as sc
import numpy as np

ModuleNotFoundError: No module named 'scanpy'

In [ ]:
# INPUT
df_class_RCTD = pd.read_csv(path + 'RCTD_filters.csv')   # RCTD results

# NUCLEI GROUPED
nuclei_grouped = sc.read_h5ad(results_folder+'/'+sample_name+'_nuclei_grouped.h5ad')

# EXPANDED NUCLEI
expanded_nuclei = sc.read_h5ad(results_folder+'/'+sample_name+'_expanded_nuclei.h5ad')



In [ ]:
# select nuclei or cells
nuclei_grouped.obs_names = nuclei_grouped.obs_names.astype(str)
nuclei_selected = (np.array(df_class_RCTD.index[df_class_RCTD['type'] == 'nuclei'])).astype(str)
nuclei_grouped_filtered = nuclei_grouped[nuclei_selected, :]
nuclei_grouped_filtered = nuclei_grouped_filtered.copy()
nuclei_grouped_filtered.obs["cell_nucleus"] = "nucleus"

expanded_nuclei.obs_names = expanded_nuclei.obs_names.astype(str)
cells_selected = (np.array(df_class_RCTD.index[df_class_RCTD['type'] == 'cells'])).astype(str)
expanded_nuclei_filtered = expanded_nuclei[cells_selected, :]
expanded_nuclei_filtered = expanded_nuclei_filtered.copy()
expanded_nuclei_filtered.obs["cell_nucleus"] = "cell"


In [ ]:
# merge nuclei and cells in adata_final
adata_final = sc.concat([nuclei_grouped_filtered, expanded_nuclei_filtered], join='outer')
adata_final.obs["cell_types"] = df_class_RCTD.loc[adata_final.obs.index, "class"]


In [ ]:
# final object on which to perform downstream analyses
adata_final.write_h5ad(results_folder + sample_name+'_adata_final.h5ad')

# 6. STANDARD CLUSTERING ANALYSIS (PYTHON)

In [ ]:
import matplotlib.image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from stardist import random_label_cmap, _draw_polygons
import pandas as pd
import numpy as np
import scanpy as sc
from scipy import sparse
import bin2cell as b2c
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import cv2
import os
import pickle
import anndata
import geopandas as gpd
from tifffile import imread, imwrite
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
import seaborn as sns
import scrublet as scr
#pip install celltypist
import celltypist
from celltypist import models
import itertools
import anndata as ad
from pandas.api.types import CategoricalDtype

In [ ]:
def up_to_pca(adata, n_top_genes=3000, from_raw=True):
    print('Top genes: '+str(n_top_genes))
    if from_raw:
        x = adata.raw.to_adata()
    else:
        x = adata.copy()
        sc.pp.normalize_total(x)
        sc.pp.log1p(x)
    if n_top_genes == 0:
        sc.pp.highly_variable_genes(x, min_mean=0.0125, max_mean=3, min_disp=-2)
    else:
        sc.pp.highly_variable_genes(x, n_top_genes=n_top_genes)
    sc.pl.highly_variable_genes(x)
    x.raw = x
    x = x[:, x.var.highly_variable]
    sc.pp.scale(x, max_value = 10)
    sc.tl.pca(x)
    sc.pl.pca_variance_ratio(x, n_pcs=50)
    return x

def from_pca_to_leiden(adata, n_neighbors = 20, n_pcs = 15, resolution = 1, plot_spatial=True):
    print('Neighbors: '+str(n_neighbors))
    print('PCA n: '+str(n_pcs))
    print('resolution: '+str(resolution))
    x = adata.copy()
    sc.pp.neighbors(x, n_neighbors = n_neighbors, n_pcs = n_pcs)
    sc.tl.umap(x)
    sc.tl.leiden(x, flavor="igraph", resolution= resolution)
    n_clusters = len(np.unique(x.obs.leiden))
    x.uns["leiden_colors"] = create_palette_matplotlib(len(x.obs.leiden.cat.categories), white=False)
    sc.pl.umap(x, color=["leiden"], legend_loc = 'on data')
    if plot_spatial:
    	sc.pl.spatial(x, color="leiden")
    return x




In [ ]:
# INPUT DATA

# adata
adata = sc.read_h5ad(results_folder + sample_name+'_adata_final.h5ad')


In [ ]:
adata = up_to_pca(adata, n_top_genes=3000, from_raw=True)
adata = from_pca_to_leiden(adata, n_neighbors = 20, n_pcs = 15, resolution = 1, plot_spatial=True)
# n_top_genes, n_neighbors, n_pcs, resolution potrebbero essere dati come input?

In [ ]:
adata.write_h5ad(results_folder + sample_name+'_adata_final.h5ad')   # overwrite

# 7. NICHE ANALYSIS (PYTHON)

In [ ]:
from sklearn.cluster import KMeans
import scanpy as sc
from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [ ]:
# INPUT DATA

# adata
adata = sc.read_h5ad(results_folder + sample_name+'_adata_final.h5ad')


In [ ]:
# find n nearest neighbours, with max threshold

n = 20 # da dare in input il numero di neighbours considerati
max_dist = 50 # dare in input max distance (non so in che ordine sia)
coordinates = adata.obs[["array_row", "array_col"]]
neighbors = NearestNeighbors(n_neighbors=n+1, metric='euclidean')
neighbors.fit(coordinates)
distances, indices = neighbors.kneighbors(coordinates)


# Exclude the first column (which is the distance to the point itself)
distances = distances[:, 1:]
indices = indices[:, 1:]

# For each cell, keep only neighbor indices where distance < max_dist
filtered_neighbors = []
for dist_row, idx_row in zip(distances, indices):
    filtered = [idx for d, idx in zip(dist_row, idx_row) if d < max_dist]
    filtered_neighbors.append(filtered)

neighbors_df = pd.DataFrame(distances, columns=[f'distance_{i+1}' for i in range(n)])
neighbors_df['neighbor_indices'] = filtered_neighbors
neighbors_df['n_neighbors_within_max_dist'] = neighbors_df['neighbor_indices'].apply(len)

adata.obs['n_neighbors_within_max_dist'] = neighbors_df['n_neighbors_within_max_dist']



In [ ]:
# CELLTYPES distribution among nerighbours

neighbor_cell_types_counts = []
cell_types = adata.obs.cell_types.unique()
col_index = adata.obs.columns.get_loc('cell_types')

for i in range(neighbors_df.shape[0]):
    idx = np.array(neighbors_df.loc[i, 'neighbor_indices'])
    neighbor_cell_types = adata.obs.iloc[idx, col_index]
    type_counts = neighbor_cell_types.value_counts()

    count_dict = {cell_type: int(type_counts.get(cell_type, 0)) for cell_type in cell_types}
    neighbor_cell_types_counts.append(count_dict)

neighbor_cell_types_df = pd.DataFrame(neighbor_cell_types_counts, index=adata.obs.index).T # cell types x cells
neighbor_cell_types_df = neighbor_cell_types_df.div(neighbor_cell_types_df.sum(axis=0), axis=1)

In [ ]:
# KMEANS ON FRACTION OF CELLTYPES IN NEIGHBOURS

k = 20  # number of niches, to be given in input

X = neighbor_cell_types_df.T  # shape: cells x cell types
kmeans = KMeans(n_clusters=k, random_state=42)
cluster_labels = kmeans.fit_predict(X)
adata.obs['neighbour_niches'] = cluster_labels.astype(str)

In [ ]:
adata.write_h5ad(results_folder + sample_name+'_adata_final.h5ad')   # overwrite